# Retrieval Augmented Generation (RAG) with Azure OpenAI and Azure AI Search

In this notebook we will showcase how to implement a Retrieval Augement Generation pipeline using Azure AI Search.

## Environment setup
Before executing the following cells, make sure to set the following environment variables in the `.env` file or export them:
* `AZURE_OPENAI_KEY`
* `AZURE_OPENAI_ENDPOINT`
* `MODEL_DEPLOYMENT_NAME`
* `EMBEDDING_DEPLOYMENT_NAME`
* `VECTOR_STORE_ADDRESS`
* `VECTOR_STORE_PASSWORD`
* `INDEX_NAME`

<br/>
<img src="../assets/keys_endpoint.png" width="800"/>

In [1]:
!pip install -q langchain==0.1.16 python-dotenv langchain_openai azure-search-documents azure-identity


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file
import os

from langchain.vectorstores import AzureSearch
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Create Index

In [3]:
openai_api_version = "2024-02-01"

embeddings = AzureOpenAIEmbeddings(
    deployment=os.getenv('EMBEDDING_DEPLOYMENT_NAME'),
    openai_api_version=openai_api_version,
)

In [4]:
from azure.search.documents.indexes.models import (
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
)

embedding_function = embeddings.embed_query

fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        filterable=True,
    ),
    SearchableField(
        name="content",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    SearchField(
        name="content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=len(embedding_function("Text")),
        vector_search_profile_name="myHnswProfile",
    ),
    SearchableField(
        name="metadata",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
]

## Create the vector store on Azure

The following vectorstore will be used to store the embeddings of the documents we will index.

To enable semantic ranking, we need to create a semantic configuration in the Azure portal.
- From Indexes on the left-navigation pane, open an index.
- Select Semantic Configurations and then select Add Semantic Configuration.
- The New Semantic Configuration page opens with options for selecting a title field, content fields, and keyword fields. Only searchable and retrievable string fields are eligible. Make sure to list content fields and keyword fields in priority order.

<br/>
<img src="../assets/semantic_ranker_ai_search.png"/>

In [27]:
index_name: str = "ai-academy"

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=os.getenv('VECTOR_STORE_ADDRESS'),
    azure_search_key=os.getenv('VECTOR_STORE_PASSWORD'),
    index_name=index_name,
    embedding_function=embedding_function,
    fields=fields,
    # needed for semantic ranking
    semantic_configuration_name = 'my-config',
)

## Populate the vectorstore

For this notebook, we've scraped some langchain's blogs (see `data/`) we will store in the vectorstore.

In [28]:
text_loader_kwargs={'autodetect_encoding': True}

loader = DirectoryLoader(
    '../data/langchain_blog_posts/',
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs=text_loader_kwargs,
    #silent_errors=True
)

documents = loader.load()

#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)
docs = text_splitter.split_documents(documents)

len(docs)

429

In [29]:
docs[0]

Document(page_content='URL: https://blog.langchain.dev/7-24-release-notes/\nTitle: [Week of 7/24] LangChain Release Notes\n\nNew in LangSmith\n\nStreaming in the playground: The playground now supports streaming! You can enter the playground by clicking on the “Open in Playground” button on the upper right hand corner of select runs.\n\nThe playground now supports streaming! You can enter the playground by clicking on the “Open in Playground” button on the upper right hand corner of select runs. Infrastructure changes to support more users: We’ve beefed up our backend. That means more of you off the waitlist soon!\n\nWe’ve beefed up our backend. That means more of you off the waitlist soon! UI and navigation enhancements: We’ve made a lot of quality-of-life improvements to the UI, mainly around the run details page. Biggest change is the trace tree is now on the left hand side. See the screenshot below for an up-to-date picture.\n\nNew in Open Source', metadata={'source': '..\\data\\la

In [30]:
vector_store.add_documents(documents=docs)

['...']

## Similarity Searches

With Azure AI Search, we can now perform similarity searches on the documents we've indexed.

### Simple Similarity search

In [31]:
docs = vector_store.search(
    query="What is langchain?",
    k=3,
    search_type="similarity",
)

for doc in docs:
    print(doc.page_content)
    print('-' * 50)

"LangChain's (the company's) goal is to make it as easy as possible to develop LLM applications"

said Harrison Chase, co-founder and CEO of LangChain.

"To that end, we realized pretty early that what was needed - and missing - wasn't just an open source tool like LangChain, but also a complementary platform for managing these new types of applications. To that end, we built LangSmith - which is usable with or without LangChain and let's users easily debug, monitor, test, evaluate, and now (with the recently launched Hub) share and collaborate on their LLM applications.”



What Are LangSmith Traces?
--------------------------------------------------
But these chains werenâ€™t really composable. Sure - we had SequentialChain, but that wasnâ€™t amazingly usable. And under the hood the other chains involved a lot of custom code, which made it tough to enforce a common interface for all chains, and ensure that all had equal levels of batch, streaming, and async support.

Today weâ€™re ex

### Similarity Search with threshold

Queries that don’t meet the threshold requirements are exluded.

In [32]:
docs = vector_store.similarity_search_with_relevance_scores(
    query="How to interact with SQL using langchain?",
    k=3,
    score_threshold=.9,
)

for doc in docs:
    print(doc)
    print('-' * 50)

(Document(page_content='The LangChain library has multiple SQL chains and even an SQL agent aimed at making interacting with data stored in SQL as easy as possible. Here are some relevant links:\n\nIntroduction\n\nMost of an enterprise’s data is traditionally stored in SQL databases. With the amount of valuable data stored there, business intelligence (BI) tools that make it easy to query and understand the data present there have risen in popularity. But what if you could just interact with a SQL database in natural language? With LLMs today, that is possible. LLMs have an understanding of SQL and are able to write it pretty well. However, there are several issues that make this a non-trivial task.\n\nThe Problems\n\nSo LLMs can write SQL - what more is needed?\n\nUnfortunately, a few things.', metadata={'source': '..\\data\\langchain_blog_posts\\blog.langchain.dev_llms-and-sql_.txt'}), 0.9001867)
--------------------------------------------------


### Hybrid Search

In a hybrid search, we can combine the similarity search with a keyword search. Vector and non-vector text fields are queried in parallel, results are merged, and top matches of the unified result set are returned.

In [33]:
docs = vector_store.hybrid_search(
    query="How to interact with SQL using langchain?",
    k=3,
)

for doc in docs:
    print(doc.page_content)
    print('-' * 50)

The LangChain library provides different tools to interact with SQL databases which can be used to build and run queries based on natural language inputs. For example, the standard SQL Toolkit draws from standard best practices that have been extensively covered in this blogpost. However, there is still room for improvement when it comes to building a custom solution and adjusting the generic tools to the specific use case. The advantage of having a plug and play toolkit contrasts with having a solution that is not flexible enough for the user to incorporate their domain-specific knowledge about the databases.
--------------------------------------------------
The LangChain library has multiple SQL chains and even an SQL agent aimed at making interacting with data stored in SQL as easy as possible. Here are some relevant links:

Introduction

Most of an enterprise’s data is traditionally stored in SQL databases. With the amount of valuable data stored there, business intelligence (BI) 

## Semantic Ranking
In Azure AI Search, semantic ranking measurably improves search relevance by using language understanding to rerank search results.

Similarly to what presented in the previous notebook, Azure AI Search uses the context or semantic meaning of a query to compute a new relevance score over preranked results. Scores range from 4 to 0 (high to low), where a higher score indicates higher relevance.


Check the [official documentation](https://learn.microsoft.com/en-us/azure/search/semantic-search-overview#how-summaries-are-scored) for more.

In [34]:
results = vector_store.semantic_hybrid_search_with_score_and_rerank(
    query="How to interact with SQL using langchain?",
    k=3,
)

for result in results:
    doc, _, ranker_score = result
    print(f"{doc.page_content}\n**********\nScore: {ranker_score}")
    print('-' * 50)

The LangChain library provides different tools to interact with SQL databases which can be used to build and run queries based on natural language inputs. For example, the standard SQL Toolkit draws from standard best practices that have been extensively covered in this blogpost. However, there is still room for improvement when it comes to building a custom solution and adjusting the generic tools to the specific use case. The advantage of having a plug and play toolkit contrasts with having a solution that is not flexible enough for the user to incorporate their domain-specific knowledge about the databases.
**********
Score: 3.4577205181121826
--------------------------------------------------
The LangChain library has multiple SQL chains and even an SQL agent aimed at making interacting with data stored in SQL as easy as possible. Here are some relevant links:

Introduction

Most of an enterprise’s data is traditionally stored in SQL databases. With the amount of valuable data stor

In [35]:
docs = vector_store.semantic_hybrid_search(
    query="How to interact with SQL using langchain?",
    k=3,
)

for doc in docs:
    print(doc.page_content)
    print('-' * 50)

The LangChain library provides different tools to interact with SQL databases which can be used to build and run queries based on natural language inputs. For example, the standard SQL Toolkit draws from standard best practices that have been extensively covered in this blogpost. However, there is still room for improvement when it comes to building a custom solution and adjusting the generic tools to the specific use case. The advantage of having a plug and play toolkit contrasts with having a solution that is not flexible enough for the user to incorporate their domain-specific knowledge about the databases.
--------------------------------------------------
The LangChain library has multiple SQL chains and even an SQL agent aimed at making interacting with data stored in SQL as easy as possible. Here are some relevant links:

Introduction

Most of an enterprise’s data is traditionally stored in SQL databases. With the amount of valuable data stored there, business intelligence (BI) 

## Build a QA Chain

In [36]:
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import PromptTemplate

llm = AzureChatOpenAI(
    deployment_name=os.getenv('MODEL_DEPLOYMENT_NAME'),
    openai_api_version=openai_api_version,
    temperature=0.,
    max_tokens=1024
)

vector_store.search_type = "similarity"
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

template = """Use the following pieces of context to answer the question at the end.
Use three sentences maximum and keep the answer as concise as possible.
Don't try to make up the answer, only use the context to answer the question.
The pieces of context refer to langchain.

Context:
{context}

Question: {question}
Helpful Answer:"""

prompt = PromptTemplate.from_template(template)

qa_chain_base = (
    RunnableParallel(
        {"context": retriever, "question": RunnablePassthrough()}
    )
    | prompt
    | llm
    | StrOutputParser()
)

question = "Explain how to connect langchain to sql. Show me the code to do that"
print(qa_chain_base.invoke(question))

The LangChain library provides tools to interact with SQL databases, including SQL chains and an SQL agent. To connect LangChain to SQL, you can use the LangChain Expression Language, which allows you to compose chains and provides streaming, batch, and async support. Here is an example of code to connect LangChain to SQL:

```
from langchain import SQLAgent

# Create an instance of the SQLAgent
agent = SQLAgent()

# Connect to the SQL database
agent.connect(database='your_database', username='your_username', password='your_password')

# Execute a SQL query
result = agent.execute_query('SELECT * FROM your_table')

# Print the result
print(result)
```


## Build a QA Chain with Reranking

In [37]:
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import PromptTemplate

llm = AzureChatOpenAI(
    deployment_name=os.getenv('MODEL_DEPLOYMENT_NAME'),
    openai_api_version=openai_api_version,
    temperature=0.,
    max_tokens=1024
)

vector_store.search_type = "semantic_hybrid"
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

template = """Use the following pieces of context to answer the question at the end.
Use three sentences maximum and keep the answer as concise as possible.
Don't try to make up the answer, only use the context to answer the question.
The pieces of context refer to langchain.

Context:
{context}

Question: {question}
Helpful Answer:"""

prompt = PromptTemplate.from_template(template)

qa_chain_base = (
    RunnableParallel(
        {"context": retriever, "question": RunnablePassthrough()}
    )
    | prompt
    | llm
    | StrOutputParser()
)

question = "Explain how to connect langchain to sql. Show me the code to do that"
print(qa_chain_base.invoke(question))

To connect LangChain to SQL, you can use the provided code snippet:

```
from langchain.agents import AgentExecutor, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.sql_database import SQLDatabase

def create_agent(db_uri, agent_type=AgentType.OPENAI_FUNCTIONS, verbose=VERBOSE_LANGCHAIN, temperature=0, model="gpt-3.5-turbo-0613"):
    db = SQLDatabase.from_uri(db_uri)
    toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=temperature))
    return create_sql_agent(
        llm=ChatOpenAI(temperature=temperature, model=model),
        toolkit=toolkit,
        verbose=verbose,
        agent_type=agent_type,
    )
```

Make sure to use your OpenAI key for this and keep it private.


## Adding sources to the response

In [38]:
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=lambda x: x["context"])
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

res = rag_chain_with_source.invoke(question)

In [39]:
print(res["answer"])

To connect LangChain to SQL, you can use the provided code snippet:

```
from langchain.agents import AgentExecutor, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.sql_database import SQLDatabase

def create_agent(db_uri, agent_type=AgentType.OPENAI_FUNCTIONS, verbose=VERBOSE_LANGCHAIN, temperature=0, model="gpt-3.5-turbo-0613"):
    db = SQLDatabase.from_uri(db_uri)
    toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=temperature))
    return create_sql_agent(
        llm=ChatOpenAI(temperature=temperature, model=model),
        toolkit=toolkit,
        verbose=verbose,
        agent_type=agent_type,
    )
```

Make sure to use your OpenAI key for this and keep it private.


In [40]:
print(res["context"])

[Document(page_content='Initializing the LangChain Agent\n\nNote: Please use your OpenAI key for this, which should be kept private.\n\nHere\'s the code to initialize the LangChain Agent and connect it to your SQL database.\n\nfrom langchain.agents import AgentExecutor, create_sql_agent from langchain.agents.agent_toolkits import SQLDatabaseToolkit from langchain.agents.agent_types import AgentType from langchain.chat_models import ChatOpenAI from langchain.llms.openai import OpenAI from langchain.sql_database import SQLDatabase def create_agent( db_uri, agent_type=AgentType.OPENAI_FUNCTIONS, verbose=VERBOSE_LANGCHAIN, temperature=0, model="gpt-3.5-turbo-0613", ): db = SQLDatabase.from_uri(db_uri) toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=temperature)) return create_sql_agent( llm=ChatOpenAI(temperature=temperature, model=model), toolkit=toolkit, verbose=verbose, agent_type=agent_type, )', metadata={'source': '..\\data\\langchain_blog_posts\\blog.langchain.dev_how-to-s

In [41]:
for doc in res["context"]:
    print(doc.metadata['source'])

..\data\langchain_blog_posts\blog.langchain.dev_how-to-safely-query-enterprise-data-with-langchain-agents-sql-openai-gretel_.txt
..\data\langchain_blog_posts\blog.langchain.dev_how-to-safely-query-enterprise-data-with-langchain-agents-sql-openai-gretel_.txt
..\data\langchain_blog_posts\blog.langchain.dev_how-to-safely-query-enterprise-data-with-langchain-agents-sql-openai-gretel_.txt
..\data\langchain_blog_posts\blog.langchain.dev_langchain-expression-language_.txt
..\data\langchain_blog_posts\blog.langchain.dev_incorporating-domain-specific-knowledge-in-sql-llm-solutions_.txt
